## Coastal Mask

Notebook environment to migrate TIF files to CF compliant CoG

In [90]:
# Optional; code formatter, installed as jupyter lab extension
#%load_ext lab_black
# Optional; code formatter, installed as jupyter notebook extension
%load_ext nb_black

<IPython.core.display.Javascript object>

### Configure OS independent paths

In [1]:
# Import standard packages
import os
import pathlib
import sys
import json5
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP4"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", "tmp", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname, "data")

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")

# directory to export result (make if not exists)
cog_dir = ds_dir.parent.joinpath("cog") # for checking CF compliancy
cog_dirs = ds_dir.parent.joinpath("cogs_28nov24") # for making all files CF compliant
cog_dir.mkdir(parents=True, exist_ok=True)

C:\Users\soest\AppData\Local\Temp\ipykernel_97388\2754644873.py:7: DeprecationWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas still uses PyGEOS by default. However, starting with version 0.14, the default will switch to Shapely. To force to use Shapely 2.0 now, you can either uninstall PyGEOS or set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In the next release, GeoPandas will switch to using Shapely by default, even if PyGEOS is installed. If you only have PyGEOS installed to get speed-ups, this switch should be smooth. However, if you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
c:\Use

In [2]:
print('Data is already CF_compliant')

Data is already CF_compliant


In [3]:
# Set up file structure for coastal flooding hazard maps
def generate_slices(num_chunks: int, chunk_size: int) -> Tuple[slice, slice]:
    """Generate slices for chunk-based iteration."""
    for i in range(num_chunks):
        yield slice(i * chunk_size, (i + 1) * chunk_size)

def get_paths(folder_structure, base_dir=''):
    """Generate paths for a folder structure defined by a dict"""
    paths = []
    for key, value in folder_structure.items():
        if isinstance(value, dict):
            paths.extend(get_paths(value, os.path.join(base_dir, key)))
        elif isinstance(value, list):
            if value:
                for item in value:
                    if item != "":
                        paths.append(os.path.join(base_dir, key, item))
            else:
                paths.append(os.path.join(base_dir, key))
        else:
            continue
    return paths

# List different types on map folders
map_types = [   'HIGH_DEFENDED_MAPS',
                'LOW_DEFENDED_MAPS',
                'UNDEFENDED_MAPS']

# List all tif files present in first folder (note: it is assumed that the same files are present in all folders)
tif_list = glob.glob(str(ds_dir.joinpath("data", map_types[0],"*.tif")))

# List the desired folder structure as a dict
# NOTE: make sure the resulting path_list (based on folder structure) matches the tif_list
folder_structure = {
    "Mean_spring_tide": [],
    "RP": ["1000", "100", "1"],
    "SLR": {
        "High_end": ["2100", "2150"],
        "SSP126": ["2100"],
        "SSP245": ["2050", "2100"],
        "SSP585": ["2030", "2050", "2100"]
    }
}

# Get list of paths for the folder structure
path_list = get_paths(folder_structure)



In [4]:
# NOTE: Not all meta_data.json files were correct and will break the loop. 
# CHECK if all meta_data 

for map_type in map_types:

    # Get list of original tif's per map_type
    tif_list = ds_dir.joinpath(map_type).glob("*.tif")
    
    for cur_tif in tif_list:
        
        meta_fpath = cur_tif.parent.joinpath('Metadata',cur_tif.name.replace('.tif','.json'))
        
        # Load meta data
        cur_meta_data = open(meta_fpath)
        cur_meta = json5.load(cur_meta_data)

        print(cur_meta)

print('All .json files are working')

{'TITLE': 'High-Defended Pan-European Flood Map of Permanent Inundation - Mean Spring Tide', 'TITLE_ABBREVIATION': 'Mean_spring_tide_HD', 'DESCRIPTION': 'This map represents the flood simulation across Europe for the mean spring tide (considering the AVISO vertical reference). The cartographic representation is in raster format and displays the maximum flood extent and depth obtained using the hydraulic model RFSM-EDA. The spatial resolution of this raster-format process-based flood map is 25 meters. The results include the effect of the highest standard of protection at the nuts2 level. The standards of protection have been provided by Vrije Universiteit Amsterdam.', 'SHORT_DESCRIPTION': 'High-defended pan-European coastal flood map associated with the mean spring tide with a spatial resolution of 25 meters.', 'INSTITUTION': "Universidad de Cantabria, Instituto de Hidráulica Ambiental de la Universidad de Cantabria 'IHCantabria' (UC-IHC)", 'PROVIDERS': {'name': "Universidad de Cantabr

In [5]:
# DO THE WORK
## Setup folder structure
# List different types on map folders
# item_type = "mosaic"  # "single" or "mosaic"
# item_properties = ["defense level", "rp", "scenarios", "time"]
# map_types = [
#     "HIGH_DEFENDED_MAPS",
#     "LOW_DEFENDED_MAPS",
#     "UNDEFENDED_MAPS",
# ]  # 3 options
# rps = ["static", "1", "100", "1000"]  # 4 options
# scenarios = ["none", "SSP126", "SSP245", "SSP585", "High_end"]  # 5 options
# times = ["2010", "2030", "2050", "2100", "2150"]  # 5 options


# # Iterate over the original tif files
# for map_type in map_types:
#     for rp in rps:
#         for scen in scenarios:
#             for time in times:
#                 if (
#                     rp == "static" and scen == "none" and time == "2010"
#                 ):  # mean spring tide
#                     tif_list = list(
#                         pathlib.Path.joinpath(
#                             cog_dirs, map_type).glob("Mean_spring_tide.tif")
#                     )
#                     cur_path = cog_dirs.joinpath("Mean_spring_tide")
#                 elif (
#                     rp != "static" and scen == "none" and time == "2010"
#                 ):  # RPs frist batchs only for 2010 (hindcast)
#                     tif_list = list(
#                         pathlib.Path.joinpath(cog_dirs, map_type, "RP", rp).glob(
#                             "*.tif"
#                         )
#                     )
#                     cur_path = os.path.join("RP", rp)
#                     ds_dir.joinpath(COLLECTION_ID, "items", map_type, "RP").mkdir(
#                         parents=True, exist_ok=True
#                     )
#                     filename = os.path.join(map_type, "RP", rp)
#                     # print(map_type, rp, scen, time)
#                 elif rp == "static" and scen != "none":  # this is for the SLR maps
#                     tif_list = list(
#                         pathlib.Path.joinpath(
#                             cog_dirs, map_type, "SLR", scen, time
#                         ).glob("*.tif")
#                     )
#                     cur_path = os.path.join("SLR", scen, time)
#                     if (
#                         len(tif_list) > 0
#                     ):  # we have data so we continue (so not for all times)
#                         ds_dir.joinpath(
#                             COLLECTION_ID, "items", map_type, "SLR", scen
#                         ).mkdir(parents=True, exist_ok=True)
#                         filename = os.path.join(map_type, "SLR", scen, time)
#                         # print(map_type, rp, scen, time)
#                     else:  # break loop if not satisfied
#                         continue
#                 else:  # break loop if not satisfied (so not for all other combinations)
#                     continue
    


#     for cur_path, cur_tif in zip([path_list[3]], [tif_list[3]]):

#         print('currently working on: '+str(cur_path)+' '+str(cur_tif))
        
#         cur_dir = pathlib.Path(os.path.join(cog_dirs,map_type,cur_path))
#         cur_dir.mkdir(parents=True,exist_ok=True)

#         fm = rioxarray.open_rasterio(
#             cur_tif, mask_and_scale=False
#         )  # .isel({"x":slice(0, 40000), "y":slice(0, 40000)})
#         fm = fm.assign_coords(band=("band", [f"B{k+1:02}" for k in range(1)])) # NOTE: hard coded to 1, because one band
#         fm = fm.to_dataset("band")

#         # chunk size 
#         chunk_size = 2**15 # 16384, which is large, but OK for int8 datatype.

#         fm_chunked = fm.chunk({"x": chunk_size, "y": chunk_size})

#         num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
#         num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)

#         # Load meta data
#         cur_meta_data = open(os.path.join(os.path.dirname(cur_tif),'Metadata',os.path.basename(cur_tif.replace('tif','json'))))
#         cur_meta = json5.load(cur_meta_data)

#         for x_slice in generate_slices(num_x_chunks, chunk_size):
#             for y_slice in generate_slices(num_y_chunks, chunk_size):
#                 chunk = fm_chunked.isel(x=x_slice, y=y_slice)

#                 chunk = chunk.assign_coords(time=pd.Timestamp(2024, 3, 18).isoformat())

#                 for var in chunk:

#                     da = chunk[var]
                    
#                     da = (
#                         da.where(da != chunk.attrs['_FillValue'],-9999)
#                         .astype("float32")
#                         .rio.write_nodata(-9999)
#                         .rio.set_spatial_dims(x_dim="x", y_dim="y")
#                     )

#                     item_name = name_block(
#                         da,
#                         storage_prefix="",
#                         name_prefix=da.name,
#                         include_band=None, 
#                         time_dim=False,
#                         x_dim="x",
#                         y_dim="y",
#                     )

#                     # hacky fix to get rid of the espg=None string
#                     if "None" in item_name:
#                         item_name = item_name.replace("None", str(rasterio.crs.CRS(da.rio.crs).to_epsg()))

#                     print(item_name)

#                     # convert to dataset
#                     dad = da.to_dataset()

#                     # add all attributes (again)
#                     for attr_name, attr_val in cur_meta.items():
#                         if attr_name == 'PROVIDERS':
#                             attr_val = json5.dumps(attr_val)
#                         if attr_name == "MEDIA_TYPE": # change media type to tiff, leave the rest as is
#                             attr_val = "IMAGE/TIFF"
#                         dad.attrs[attr_name] = attr_val

#                     dad.attrs['Conventions'] = "CF-1.8"

#                     # make parent dir if not exists

#                     outpath = cur_dir.joinpath(item_name)
#                     outpath.parent.mkdir(parents=True, exist_ok=True)

#                     # export file
#                     dad.rio.to_raster(outpath, compress="DEFLATE", driver="COG")

#                     # set overwrite is false because tifs should be unique
#                     # try:
#                     #     write_cog(da, fname=outpath, overwrite=False).compute()
#                     # except OSError as e:
#                     #     continue


In [6]:
import re

def extract_info(file_path: Path):

    rps = ["static", "1", "100", "1000"]  # 4 options
    scenarios = ["none", "SSP126", "SSP245", "SSP585", "High_End"]  # 5 options
    times = ["2010", "2030", "2050", "2100", "2150"]  # 5 options

    # Extract the map type (the first part of the path)
    map_type = file_path.parts[0]
    
    # Extract the filename (last part of the path)
    file_name = file_path.parts[-1]
    
    # Initialize variables to None
    rp = rps[0]
    scenario = None
    time = times[0]

    # Check for RP in the filename (if any)
    if "RP1000" in file_name:
        rp = "1000"
    elif "RP100" in file_name:
        rp = "100"
    elif "RP1" in file_name:
        rp = "1"
    
    # Check for scenario in the filename (if any)
    # Normalize the filename for case-insensitive matching
    normalized_file_name = file_name.lower()
    for s in scenarios:
        pattern = re.escape(s.lower()).replace("_", "[-_]*")  # Match variations like "High_End" or "High-End"
        if re.search(pattern, normalized_file_name):
            scenario = s
            break  # stop at the first match
    
    # Check for time in the filename (if any)
    for t in reversed(times):  # Reverse the order to prioritize later matches
        if t in file_name:
            time = t
            break  # stop at the first match
    
    return str(rp), str(scenario), str(time)



In [7]:
item_type = "mosaic"  # "single" or "mosaic"
item_properties = ["defense level", "rp", "scenarios", "time"]
map_types = [
    "HIGH_DEFENDED_MAPS",
    "LOW_DEFENDED_MAPS",
    "UNDEFENDED_MAPS",
]  # 3 options

for map_type in map_types:
    print(map_type)

    # Iterate over the original tif files
    tif_list = ds_dir.joinpath(map_type).glob("*.tif")
    
    for cur_tif in tif_list:

        rp,scenario,time = extract_info(Path(*cur_tif.parts[5:]))

        print('currently working on: '+str(cur_tif))
        
        out_dir = pathlib.Path(cog_dirs.joinpath(map_type,rp,scenario,time))
        out_dir.mkdir(parents=True,exist_ok=True)

        print('writing chunks to: ' + str(out_dir))
        print(' ')

        fm = rio.open_rasterio(
            cur_tif, mask_and_scale=False
        )  # .isel({"x":slice(0, 40000), "y":slice(0, 40000)})
        fm = fm.assign_coords(band=("band", [f"B{k+1:02}" for k in range(1)])) # NOTE: hard coded to 1, because one band
        fm = fm.to_dataset("band")

        # chunk size 
        chunk_size = 2**15 # 16384, which is large, but OK for int8 datatype.

        fm_chunked = fm.chunk({"x": chunk_size, "y": chunk_size})

        num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
        num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)

        # Load meta data
        cur_meta_data = open(ds_dir.joinpath(map_type,'metadata',cur_tif.name.replace('.tif','.json')))
        cur_meta = json5.load(cur_meta_data)

        for x_slice in generate_slices(num_x_chunks, chunk_size):
            for y_slice in generate_slices(num_y_chunks, chunk_size):
                chunk = fm_chunked.isel(x=x_slice, y=y_slice)

                chunk = chunk.assign_coords(time=pd.Timestamp(2024, 3, 18).isoformat())

                for var in chunk:

                    da = chunk[var]
                    
                    da = (
                        da.where(da != chunk.attrs['_FillValue'],-9999)
                        .astype("float32")
                        .rio.write_nodata(-9999)
                        .rio.set_spatial_dims(x_dim="x", y_dim="y")
                    )

                    item_name = name_block(
                        da,
                        storage_prefix="",
                        name_prefix=da.name,
                        include_band=None, 
                        time_dim=False,
                        x_dim="x",
                        y_dim="y",
                    )

                    # hacky fix to get rid of the espg=None string
                    if "None" in item_name:
                        item_name = item_name.replace("None", str(rasterio.crs.CRS(da.rio.crs).to_epsg()))

                    print(item_name)

                    # convert to dataset
                    dad = da.to_dataset()

                    # add all attributes (again)
                    for attr_name, attr_val in cur_meta.items():
                        if attr_name == 'PROVIDERS':
                            attr_val = json5.dumps(attr_val)
                        if attr_name == "MEDIA_TYPE": # change media type to tiff, leave the rest as is
                            attr_val = "IMAGE/TIFF"
                        dad.attrs[attr_name] = attr_val

                    dad.attrs['Conventions'] = "CF-1.8"

                    # make parent dir if not exists

                    outpath = out_dir.joinpath(item_name)
                    outpath.parent.mkdir(parents=True, exist_ok=True)

                    # export file
                    dad.rio.to_raster(outpath, compress="DEFLATE", driver="COG")

                    del dad, da

                del chunk

        del fm
                    # set overwrite is false because tifs should be unique
                    # try:
                    #     write_cog(da, fname=outpath, overwrite=False).compute()
                    # except OSError as e:
                    #     continue

HIGH_DEFENDED_MAPS
currently working on: P:\11207608-coclico\FULLTRACK_DATA\WP4\data\HIGH_DEFENDED_MAPS\Mean_spring_tide_HD.tif
writing chunks to: P:\11207608-coclico\FULLTRACK_DATA\WP4\cogs_28nov24\HIGH_DEFENDED_MAPS\static\None\2010
 


C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:38: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:39: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)


B01_epsg=3035_x=2924487_y=4596737.tif
B01_epsg=3035_x=2924487_y=3777537.tif
B01_epsg=3035_x=2924487_y=2958337.tif
B01_epsg=3035_x=2924487_y=2139137.tif
B01_epsg=3035_x=2924487_y=1385887.tif
B01_epsg=3035_x=3743687_y=4596737.tif
B01_epsg=3035_x=3743687_y=3777537.tif
B01_epsg=3035_x=3743687_y=2958337.tif
B01_epsg=3035_x=3743687_y=2139137.tif
B01_epsg=3035_x=3743687_y=1385887.tif
B01_epsg=3035_x=4562887_y=4596737.tif
B01_epsg=3035_x=4562887_y=3777537.tif
B01_epsg=3035_x=4562887_y=2958337.tif
B01_epsg=3035_x=4562887_y=2139137.tif
B01_epsg=3035_x=4562887_y=1385887.tif
B01_epsg=3035_x=5382087_y=4596737.tif
B01_epsg=3035_x=5382087_y=3777537.tif
B01_epsg=3035_x=5382087_y=2958337.tif
B01_epsg=3035_x=5382087_y=2139137.tif
B01_epsg=3035_x=5382087_y=1385887.tif
B01_epsg=3035_x=6201287_y=4596737.tif
B01_epsg=3035_x=6201287_y=3777537.tif
B01_epsg=3035_x=6201287_y=2958337.tif
B01_epsg=3035_x=6201287_y=2139137.tif
B01_epsg=3035_x=6201287_y=1385887.tif
currently working on: P:\11207608-coclico\FULLTRAC

C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:38: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:39: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)


B01_epsg=3035_x=1547762_y=4596462.tif
B01_epsg=3035_x=1547762_y=3777262.tif
B01_epsg=3035_x=1547762_y=2958062.tif
B01_epsg=3035_x=1547762_y=2138862.tif
B01_epsg=3035_x=1547762_y=1319662.tif
B01_epsg=3035_x=1547762_y=941637.tif
B01_epsg=3035_x=2366962_y=4596462.tif
B01_epsg=3035_x=2366962_y=3777262.tif
B01_epsg=3035_x=2366962_y=2958062.tif
B01_epsg=3035_x=2366962_y=2138862.tif
B01_epsg=3035_x=2366962_y=1319662.tif
B01_epsg=3035_x=2366962_y=941637.tif
B01_epsg=3035_x=3186162_y=4596462.tif
B01_epsg=3035_x=3186162_y=3777262.tif
B01_epsg=3035_x=3186162_y=2958062.tif
B01_epsg=3035_x=3186162_y=2138862.tif
B01_epsg=3035_x=3186162_y=1319662.tif
B01_epsg=3035_x=3186162_y=941637.tif
B01_epsg=3035_x=4005362_y=4596462.tif
B01_epsg=3035_x=4005362_y=3777262.tif
B01_epsg=3035_x=4005362_y=2958062.tif
B01_epsg=3035_x=4005362_y=2138862.tif
B01_epsg=3035_x=4005362_y=1319662.tif
B01_epsg=3035_x=4005362_y=941637.tif
B01_epsg=3035_x=4824562_y=4596462.tif
B01_epsg=3035_x=4824562_y=3777262.tif
B01_epsg=3035_x=

...

C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:38: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:39: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)


B01_epsg=3035_x=1547762_y=4596737.tif
B01_epsg=3035_x=1547762_y=3777537.tif
B01_epsg=3035_x=1547762_y=2958337.tif
B01_epsg=3035_x=1547762_y=2139137.tif
B01_epsg=3035_x=1547762_y=1319937.tif
B01_epsg=3035_x=1547762_y=941637.tif
B01_epsg=3035_x=2366962_y=4596737.tif
B01_epsg=3035_x=2366962_y=3777537.tif
B01_epsg=3035_x=2366962_y=2958337.tif
B01_epsg=3035_x=2366962_y=2139137.tif
B01_epsg=3035_x=2366962_y=1319937.tif
B01_epsg=3035_x=2366962_y=941637.tif
B01_epsg=3035_x=3186162_y=4596737.tif
B01_epsg=3035_x=3186162_y=3777537.tif
B01_epsg=3035_x=3186162_y=2958337.tif
B01_epsg=3035_x=3186162_y=2139137.tif
B01_epsg=3035_x=3186162_y=1319937.tif
B01_epsg=3035_x=3186162_y=941637.tif
B01_epsg=3035_x=4005362_y=4596737.tif
B01_epsg=3035_x=4005362_y=3777537.tif
B01_epsg=3035_x=4005362_y=2958337.tif
B01_epsg=3035_x=4005362_y=2139137.tif
B01_epsg=3035_x=4005362_y=1319937.tif
B01_epsg=3035_x=4005362_y=941637.tif
B01_epsg=3035_x=4824562_y=4596737.tif
B01_epsg=3035_x=4824562_y=3777537.tif
B01_epsg=3035_x=

C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:38: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_x_chunks = math.ceil(fm_chunked.dims["x"] / chunk_size)
C:\Users\soest\AppData\Local\Temp\ipykernel_97388\1936282171.py:39: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  num_y_chunks = math.ceil(fm_chunked.dims["y"] / chunk_size)


B01_epsg=3035_x=1547762_y=4596737.tif
B01_epsg=3035_x=1547762_y=3777537.tif
B01_epsg=3035_x=1547762_y=2958337.tif
B01_epsg=3035_x=1547762_y=2139137.tif
B01_epsg=3035_x=1547762_y=1319937.tif
B01_epsg=3035_x=1547762_y=941637.tif
B01_epsg=3035_x=2366962_y=4596737.tif
B01_epsg=3035_x=2366962_y=3777537.tif
B01_epsg=3035_x=2366962_y=2958337.tif
B01_epsg=3035_x=2366962_y=2139137.tif
B01_epsg=3035_x=2366962_y=1319937.tif


MemoryError: Unable to allocate 4.00 GiB for an array with shape (32768, 32768) and data type float32

In [72]:
cur_meta

{'TITLE': 'High-Defended Pan-European Flood Map of Permanent Inundation - Mean Spring Tide',
 'TITLE_ABBREVIATION': 'Mean_spring_tide_HD',
 'DESCRIPTION': 'This map represents the flood simulation across Europe for the mean spring tide (considering the AVISO vertical reference). The cartographic representation is in raster format and displays the maximum flood extent and depth obtained using the hydraulic model RFSM-EDA. The spatial resolution of this raster-format process-based flood map is 25 meters. The results include the effect of the highest standard of protection at the nuts2 level. The standards of protection have been provided by Vrije Universiteit Amsterdam.',
 'SHORT_DESCRIPTION': 'High-defended pan-European coastal flood map associated with the mean spring tide with a spatial resolution of 25 meters.',
 'INSTITUTION': "Universidad de Cantabria, Instituto de Hidráulica Ambiental de la Universidad de Cantabria 'IHCantabria' (UC-IHC)",
 'PROVIDERS': {'name': "Universidad de Ca

In [50]:
map_types = [
    "HIGH_DEFENDED_MAPS",
    "LOW_DEFENDED_MAPS",
    "UNDEFENDED_MAPS",
]  # 3 options

for map in map_types:
    print(map_type)

UNDEFENDED_MAPS
UNDEFENDED_MAPS
UNDEFENDED_MAPS


In [55]:
map_type

'HIGH_DEFENDED_MAPS'

In [23]:
for i in ds_dir.glob("**/*.tif"):

    relative_path = Path(*i.parts[5:])
    print(relative_path)



Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\Mean_spring_tide_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\RP1000_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\RP100_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\RP1_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_High-End_2100_subs_2050_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_High-End_2150_subs_2050_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP126_2100_subs_2050_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP245_2050_subs_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP245_2100_subs_2050_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP585_2030_subs_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP585_2050_subs_HD.tif
Deliverable_2024_Feb\HIGH_DEFENDED_MAPS\SLR_SSP585_2100_subs_2050_HD.tif
Deliverable_2024_Feb\LOW_DEFENDED_MAPS\Mean_spring_tide_LD.tif
Deliverable_2024_Feb\LOW_DEFENDED_MAPS\RP1000_LD.tif
Deliverable_2024_Feb\LOW_DEFENDED_MAPS\RP100_LD.tif
Deliverable_2024_Feb\LOW_DEFENDED_MAP